In [2]:
!pip install ctransformers PyPDF2


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.9/9.9 MB 77.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 232.6/232.6 kB 17.4 MB/s eta 0:00:00


In [3]:
!pip install pymupdf

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 19.6/19.6 MB 70.8 MB/s eta 0:00:00


In [4]:
!git clone https://github.com/Rakeshdevara/LLM_App.git

Cloning into 'LLM_App'...
remote: Enumerating objects: 26, done.
remote: Counting objects: 100% (26/26), done.
remote: Compressing objects: 100% (23/23), done.
remote: Total 26 (delta 11), reused 0 (delta 0), pack-reused 0 (from 0)
Receiving objects: 100% (26/26), 7.22 KiB | 3.61 MiB/s, done.
Resolving deltas: 100% (11/11), done.


In [5]:
cd /content/LLM_App/local-ChatBot/

/content/LLM_App/local-ChatBot


In [6]:
!ls

README.md  requirements.txt  solutions


In [7]:
!pip install -r requirements.txt

  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 83.5/83.5 kB 6.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.6/57.6 kB 5.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 22.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.8/60.8 kB 5.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 163.8/163.8 kB 11.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.5/4.5 MB 35.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.0/92.0 kB 8.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 170.9/170.9 kB 13.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 228.7/228.7 kB 16.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.4/5.4 MB 23.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 4.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 74.5/7

In [7]:
import os
import fitz  # PyMuPDF for PDF reading
from ctransformers import AutoModelForCausalLM

In [2]:
# Load LLM model
llm = AutoModelForCausalLM.from_pretrained(
    "zoltanctoth/orca_mini_3B-GGUF", model_file="orca-mini-3b.q4_0.gguf"
)


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Fetching 0 files: 0it [00:00, ?it/s]

Fetching 1 files:   0%|          | 0/1 [00:00<?, ?it/s]

orca-mini-3b.q4_0.gguf:   0%|          | 0.00/1.93G [00:00<?, ?B/s]

In [8]:
!ls

LLM_App  resumes  sample_data


In [9]:
# Folder containing resumes
RESUME_FOLDER = "resumes"

In [10]:
# Function to extract text from a PDF
def extract_text_from_pdf(pdf_path):
    with fitz.open(pdf_path) as doc:
        text = ""
        for page in doc:
            text += page.get_text()
    return text

In [11]:
# Function to check if the resume is related to the given role (e.g., Data Engineer, Software Engineer)
def is_role_related_resume(text, role_keywords):
    return any(keyword.lower() in text.lower() for keyword in role_keywords)


In [12]:
# Function to summarize the resume using LLM
def summarize_resume(text, role):
    prompt = f"Summarize the following resume focusing on {role} skills:\n{text}"
    summary = llm(prompt)  # Summarization using the LLM
    return summary


In [13]:
# Function to extract name from resume text (basic example)
def extract_name(text):
    # Simple heuristic to find the name (you can improve this with NLP techniques)
    lines = text.splitlines()
    for line in lines:
        if "Name:" in line:
            return line.split("Name:")[-1].strip()
    return "Unknown"

In [14]:
# Process all resumes in the folder
def process_resumes(role_keywords, role_name):
    results = []
    for file_name in os.listdir(RESUME_FOLDER):
        if file_name.endswith(".pdf"):
            pdf_path = os.path.join(RESUME_FOLDER, file_name)
            text = extract_text_from_pdf(pdf_path)

            if is_role_related_resume(text, role_keywords):
                name = extract_name(text)
                summary = summarize_resume(text, role_name)
                results.append({
                    "file_name": file_name,
                    "name": name,
                    "summary": summary
                })

    return results

In [15]:
# Example usage:
# Define role-specific keywords
role_name = "Software Engineer"  # or "Data Engineer"
role_keywords = ["Software Engineer", "Java", "Spring Boot", "Microservices", "REST APIs"]  # Add relevant keywords for Software Engineer

# Run the resume processing for the given role
results = process_resumes(role_keywords, role_name)

In [ ]:
role_name = "Data Engineer"
role_keywords = ["Data Engineer", "ETL", "Apache Spark", "Big Data", "SQL", "Python"]


In [16]:
# Print the results
for result in results:
    print(f"PDF File: {result['file_name']}")
    print(f"Name: {result['name']}")
    print(f"Summary: {result['summary']}")
    print("="*40)

PDF File: John_Smith_resume.pdf
Name: John Smith
Summary: - Third-party service integrations
Experience with:
- Git
- Continuous Integration and Delivery (CI/CD)
- AWS Cloud
- Docker
- Kubernetes
- MySQL
- PostgreSQL
Software Engineer (2018-2020)
- Developed web applications using React, Node.js, and MongoDB.
- Implemented responsive designs using CSS and Bootstrap.
- Conducted debugging and troubleshooting for various projects.
- Collaborated with other developers to deliver projects on time.
- Familiarized with Agile methodologies such as Scrum and Kanban.
- Familiarity with Webpack and Nginx servers.
- Worked with JIRA for project management.
Education:
Masters in Computer Science (2018)
- Graduated with a CGPA of 3.9 out of 4.0
Bachelor of Science in Computer Science (2016)
- Graduated with honors with a CGPA of 3.7 out of 4.0


In [17]:
# Model evaluation

In [18]:
from sklearn.metrics import accuracy_score

def calculate_keyword_accuracy(summary, original_text, role_keywords):
    summary_keywords = [kw for kw in role_keywords if kw.lower() in summary.lower()]
    original_keywords = [kw for kw in role_keywords if kw.lower() in original_text.lower()]

    # Calculate accuracy based on matched keywords
    accuracy = len(set(summary_keywords).intersection(set(original_keywords))) / len(original_keywords) if original_keywords else 0
    return accuracy
